## Librairies

In [67]:
import numpy as np
import tarfile
import os
import pandas as pd
import io
import matplotlib.pyplot as plt
!pip install pandas_read_xml
import pandas_read_xml as pdx
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from lxml import objectify
from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Dataset

In [144]:
from google.colab import drive
drive.mount('/content/drive')
data_folder = "/content/drive/MyDrive/ADA/Project/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
#Movie metadata
df_movie = pd.read_table(data_folder +'movie.metadata.tsv',header=None)
df_movie.columns=['Wikipedia_movie_ID' , 'Freebase_movie_ID', 'Movie_name' , 'Movie_date' , 'Movie_revenue' , 'Movie_runtime' , 'Movie_languages' , 'Movie_countries' , 'Movie_genres']
df_movie

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_date,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [146]:
#Character metadata
df_character = pd.read_table(data_folder +'character.metadata.tsv',header=None)
df_character.columns=['Wikipedia_movie_ID' , 'Freebase_movie_ID' , 'Movie_date' , 'Character_name' , 'Actor_date_of_birth' , 'Actor_gender' , 'Actor_height' , 'Actor_ethnicity' , 'Actor_name' , 'Actor_age_at_movie_release' , 'Freebase_character_actor_ID' , 'Freebase_character_ID' , 'Freebase_actor_ID'  ]
df_character


,Wikipedia_movie_ID,Freebase_movie_ID,Movie_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character_actor_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm




---
*Transformation .txt to dataframe*

--- 

## Part to add

In [147]:
####Here the goal is to find the dataset of all countries where are movies done
# !!!! Some movies may have been shoot in different countries !!!
#Here when a movie appears in two countries, it is count for one movie in each country

#Initializating data
find_countries = df_movie['Movie_countries'].str.replace('{|}|"', '').str.split(':|,', expand =True)
df_id_countries = find_countries
df_name_countries = find_countries

#Gather all id and coutries from dataset
for i in range(14):
  df_name_countries = df_name_countries.drop(columns = 2*i,axis=1) #split all countries 
  df_id_countries = df_id_countries.drop(columns = 2*i+1,axis=1) #split all id 
list_name_countries = []
list_id_countries = []
for col in range(14):
  colist_id = df_id_countries[df_id_countries.columns[col]].values.tolist() # transform all colums of id dataframe in list
  colist_name = df_name_countries[df_name_countries.columns[col]].values.tolist() #
  for val_id in colist_id:
    list_id_countries.append(val_id) #regroup all id 
  for val_name in colist_name:
    list_name_countries.append(val_name) #regroup all countries

#Final dataframe for countries
Iter_countries = pd.DataFrame(list(zip(list_id_countries,list_name_countries)), columns= ['Countries_ID','Countries']).dropna()

#numbers_movies_by_country = df_countries.groupby('Countries').count()

df_countries = pd.DataFrame(Iter_countries.groupby(['Countries', 'Countries_ID']).count().reset_index())

#Number of women in movies in the dataset
actress = df_character[df_character['Actor_gender']=='F']
actor = df_character[df_character['Actor_gender']=='M']


df_act_by_country= df_countries
 
actress_count=[]
actor_count=[]
ss= 0
for country in df_act_by_country['Countries_ID']:
  count_number_actress_bymovie =actress['Wikipedia_movie_ID'].isin(df_movie['Wikipedia_movie_ID'].where(df_movie['Movie_countries'].str.contains(str(country)))) ==True
  actress_count.append(count_number_actress_bymovie.sum())

  count_number_actor_bymovie =actor['Wikipedia_movie_ID'].isin(df_movie['Wikipedia_movie_ID'].where(df_movie['Movie_countries'].str.contains(str(country)))) ==True
  actor_count.append(count_number_actor_bymovie.sum())

df_act_by_country['Number_Acting_Male']= np.array(actor_count)
df_act_by_country['Number_Acting_Female']= np.array(actress_count)
df_act_by_country = df_act_by_country.replace(0, np.nan)
df_act_by_country = df_act_by_country.dropna( axis=0).reset_index().drop(['index'], axis=1)
df_act_by_country

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning:

The default value of regex will change from True to False in a future version.



,Countries,Countries_ID,Number_Acting_Male,Number_Acting_Female
0,Afghanistan,/m/0jdd,34.0,23.0
1,Albania,/m/0jdx,46.0,17.0
2,Algeria,/m/0h3y,89.0,32.0
3,Argentina,/m/0jgd,1503.0,797.0
4,Armenia,/m/0jgx,11.0,8.0
...,...,...,...,...
124,Weimar Republic,/m/084n_,235.0,100.0
125,West Germany,/m/082fr,1896.0,945.0
126,Yugoslavia,/m/087vz,815.0,272.0
127,Zambia,/m/088vb,11.0,5.0


In [148]:
g = df_act_by_country['Number_Acting_Female'].sum()/(df_act_by_country['Number_Acting_Female'].sum()+df_act_by_country['Number_Acting_Male'].sum())
g

0.33208812833518636

In [149]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px # for visualization 
import plotly.offline as py 
import plotly.graph_objs as go 
from plotly.figure_factory import create_table # for creating nice table 

rr = df_act_by_country
rr['Total_Act']= rr['Number_Acting_Male'] +rr['Number_Acting_Female']


fig1 = px.scatter(rr, x='Number_Acting_Male', y='Number_Acting_Female',hover_name=rr['Countries'], size='Total_Act',size_max=60)
fig1.show()
fig2 = px.scatter(rr, x='Number_Acting_Male', y='Number_Acting_Female',hover_name=rr['Countries'], log_x=True,log_y=True, range_x=[100, 190000],range_y=[100, 180000], size='Total_Act',size_max=60)
fig2.show() 

In [150]:
#Evolution of usa actress during time
df_usa = df_movie.loc[df_movie['Movie_countries'].explode().isin(['/m/09c7w0'])[df_movie['Movie_countries'].explode().isin(['/m/09c7w0'])].index]
df_character_usa = df_character.merge(df_usa['Freebase_movie_ID'], on='Freebase_movie_ID')

#Recupere les actrices
usa_actress  = df_character[df_character['Actor_gender']=='F']
usa_actress_with_nodate  = df_character[df_character['Actor_gender']=='F'].dropna(subset =['Movie_date'])

print('There are '+str(len(usa_actress)- len(usa_actress_with_nodate))+' with no movie date for actress')

#Recupere les acteurs
usa_actor  = df_character[df_character['Actor_gender']=='M']
usa_actor_with_nodate  = df_character[df_character['Actor_gender']=='M'].dropna(subset =['Movie_date'])

print('There are '+str(len(usa_actor)- len(usa_actor_with_nodate))+' with no movie date for actor')


There are 3491 with no movie date for actress
There are 6355 with no movie date for actor


In [151]:
t1 = usa_actor_with_nodate.sort_values('Movie_date')
t2 = usa_actress_with_nodate.sort_values('Movie_date')

t1 = t1.loc[t1['Movie_date']!= "1010-12-02"]
t2 = t2.loc[t2['Movie_date']!= "1010-12-02"]

t1['Movie_date'] = pd.to_datetime(t1['Movie_date'])
t2['Movie_date'] = pd.to_datetime(t2['Movie_date'])

r1 = t1.groupby(pd.Grouper(key='Movie_date', axis=0, freq='Y')).count()
r1 = r1.loc[r1['Wikipedia_movie_ID']!=0]

r2 = t2.groupby(pd.Grouper(key='Movie_date', axis=0, freq='Y')).count()
r2 = r2.loc[r2['Wikipedia_movie_ID']!=0]

In [152]:
h =r1.merge(r2,left_on='Movie_date', right_on='Movie_date').reset_index()
h

,Movie_date,Wikipedia_movie_ID_x,Freebase_movie_ID_x,Character_name_x,Actor_date_of_birth_x,Actor_gender_x,Actor_height_x,Actor_ethnicity_x,Actor_name_x,Actor_age_at_movie_release_x,...,Character_name_y,Actor_date_of_birth_y,Actor_gender_y,Actor_height_y,Actor_ethnicity_y,Actor_name_y,Actor_age_at_movie_release_y,Freebase_character_actor_ID_y,Freebase_character_ID_y,Freebase_actor_ID_y
0,1888-12-31,2,2,0,0,2,0,0,2,0,...,0,0,2,0,0,2,0,2,0,2
1,1894-12-31,3,3,0,3,3,0,1,3,0,...,0,1,1,0,0,1,0,1,0,1
2,1896-12-31,4,4,2,4,4,1,1,4,0,...,1,4,5,0,0,5,0,5,1,5
3,1897-12-31,1,1,0,1,1,0,0,1,0,...,0,0,2,0,0,2,0,2,0,2
4,1898-12-31,3,3,0,3,3,0,0,3,0,...,1,1,1,0,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2012-12-31,4502,4502,2915,3420,4502,2101,1472,4502,3244,...,1690,1928,2545,1274,783,2545,1836,2545,1690,2545
117,2013-12-31,643,643,391,580,643,468,298,643,572,...,144,260,281,210,141,281,254,281,144,281
118,2014-12-31,21,21,6,21,21,16,13,21,20,...,4,14,15,10,8,15,14,15,4,15
119,2015-12-31,12,12,7,11,12,9,9,12,11,...,2,2,2,2,1,2,2,2,2,2


In [153]:
fig = px.line(x=h['Movie_date'], y=[h['Wikipedia_movie_ID_x'],h['Wikipedia_movie_ID_y']])
fig.show()
